# Prefect Workflow


In [12]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

import re

from prefect import task, Flow

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stem = PorterStemmer()
lemma = WordNetLemmatizer()

# Loading the data using the below function:

In [13]:
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    df = pd.read_csv(file_path)
    df['label'] = df['Ratings'].apply(lambda x: 'negative' if x <= 2 else 'positive')
    return df

# Function to Identify Input and Output variables

In [14]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y

# Train-Test Split

In [15]:
def split_train_test(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# CountVectorizer

In [16]:
def preprocess_data(X_train, X_test):
    """
    Preprocess the text data and transform it into numerical using CountVectorizer.
    """
    def preprocess_text(text):
        if isinstance(text, str):
            # removes special characters
            text = re.sub("[^a-zA-Z]"," ", text)
            
            # converts words to lowercase
            text = text.lower()
            
            # tokenization
            text = text.split()
            
            # removes the stop words
            text = [word for word in text if word not in stopwords.words('english')]
            
            # applying lemmatization
            text = [lemma.lemmatize(word) for word in text]
            
            return " ".join(text)
        else:
            return ""  # Replace non-string values with an empty string
    
    # Preprocess training data
    X_train_preprocessed = [preprocess_text(text) for text in X_train]
    
    # Preprocess testing data
    X_test_preprocessed = [preprocess_text(text) for text in X_test]
    
    # Transform text data into numerical using CountVectorizer
    cv = CountVectorizer()
    X_train_vectorized = cv.fit_transform(X_train_preprocessed)
    X_test_vectorized = cv.transform(X_test_preprocessed)
    
    return X_train_vectorized, X_test_vectorized

In [17]:
def train_model(X_train_vectorized, y_train):
    """
    Train the machine learning model.
    """
    nv_model =  MultinomialNB()
    nv_model.fit(X_train_vectorized, y_train)
    return nv_model

In [18]:
def evaluate_model(model, X_train_vectorized, y_train,X_test_vectorized , y_test):
    """
    Evaluating the model.
    
    """
    from sklearn import metrics
    y_train_pred = model.predict(X_train_vectorized)
    y_test_pred = model.predict(X_test_vectorized)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [19]:
def workflow(data_path):
    DATA_PATH = "data.csv"

    # Load data
    df = load_data(DATA_PATH)
    
    # Identify Inputs and Output
    X, y = split_inputs_output(df, "Review text", "label")

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Preprocess the data and transform it into numerical using CountVectorizer
    X_train_vectorized, X_test_vectorized = preprocess_data(X_train, X_test)
    
    # Build a model
    model = train_model(X_train_vectorized, y_train)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_vectorized, y_train, X_test_vectorized, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [20]:
if __name__ == "__main__":
    workflow(data_path="data.csv")

Train Score: 0.9365998747651847
Test Score: 0.9183098591549296


## Building  Prefect Workflow


In [21]:
pip install prefect

Note: you may need to restart the kernel to use updated packages.


In [11]:
from prefect import task, flow

In [22]:
@task
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    df = pd.read_csv(file_path)
    df['label'] = df['Ratings'].apply(lambda x: 'negative' if x <= 2 else 'positive')
    return df

@task
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y

@task
def split_train_test(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

@task
def preprocess_data(X_train, X_test):
    """
    Preprocess the text data and transform it into numerical using CountVectorizer.
    """
    def preprocess_text(text):
        if isinstance(text, str):
            # removes special characters
            text = re.sub("[^a-zA-Z]"," ", text)
            
            # converts words to lowercase
            text = text.lower()
            
            # tokenization
            text = text.split()
            
            # removes the stop words
            text = [word for word in text if word not in stopwords.words('english')]
            
            # applying lemmatization
            text = [lemma.lemmatize(word) for word in text]
            
            return " ".join(text)
        else:
            return ""  # Replace non-string values with an empty string
    
    # Preprocess training data
    X_train_preprocessed = [preprocess_text(text) for text in X_train]
    
    # Preprocess testing data
    X_test_preprocessed = [preprocess_text(text) for text in X_test]
    
    # Transform text data into numerical using CountVectorizer
    cv = CountVectorizer()
    X_train_vectorized = cv.fit_transform(X_train_preprocessed)
    X_test_vectorized = cv.transform(X_test_preprocessed)
    
    return X_train_vectorized, X_test_vectorized


@task
def train_model(X_train_vectorized, y_train):
    """
    Train the machine learning model.
    """
    nv_model =  MultinomialNB()
    nv_model.fit(X_train_vectorized, y_train)
    return nv_model

@task
def evaluate_model(model, X_train_vectorized, y_train,X_test_vectorized , y_test):
    """
    Evaluating the model.
    
    """
    from sklearn import metrics
    y_train_pred = model.predict(X_train_vectorized)
    y_test_pred = model.predict(X_test_vectorized)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [25]:
@flow(name="Naive_bayes_model_development")
def workflow(data_path):
    DATA_PATH = "data.csv"

    # Load data
    df = load_data(DATA_PATH)
    
    # Identify Inputs and Output
    X, y = split_inputs_output(df, "Review text", "label")

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Preprocess the data and transform it into numerical using CountVectorizer
    X_train_vectorized, X_test_vectorized= preprocess_data(X_train, X_test)
    
    # Build a model
    model = train_model(X_train_vectorized, y_train)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_vectorized, y_train, X_test_vectorized, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

/Users/prashantheleshala/anaconda3/lib/python3.11/site-packages/prefect/flows.py:357: UserWarning: A flow named 'Naive_bayes_model_development' and defined at '/var/folders/kw/ny9zsgd901d6kgbrsz52jj7h0000gn/T/ipykernel_8037/25305068.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [26]:
if __name__ == "__main__":
    data_path = "data.csv"  # Path to your data file
    workflow(data_path)


14:26:48.009 | INFO    | prefect.engine - Created flow run 'attractive-capuchin' for flow 'Naive_bayes_model_development'

14:26:48.066 | INFO    | Flow run 'attractive-capuchin' - Created task run 'load_data-0' for task 'load_data'

14:26:48.067 | INFO    | Flow run 'attractive-capuchin' - Executing 'load_data-0' immediately...

14:26:48.143 | INFO    | Task run 'load_data-0' - Finished in state Completed()

14:26:48.162 | INFO    | Flow run 'attractive-capuchin' - Created task run 'split_inputs_output-0' for task 'split_inputs_output'

...

14:26:51.934 | INFO    | Flow run 'attractive-capuchin' - Created task run 'evaluate_model-0' for task 'evaluate_model'

14:26:51.935 | INFO    | Flow run 'attractive-capuchin' - Executing 'evaluate_model-0' immediately...

14:26:52.000 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.9365998747651847
Test Score: 0.9183098591549296


14:26:52.024 | INFO    | Flow run 'attractive-capuchin' - Finished in state Completed('All states completed.')

In [27]:
if __name__ == "__main__":
    workflow.serve(
        name="model_prefect_integration",
        cron="* * * * *"
    )

/var/folders/kw/ny9zsgd901d6kgbrsz52jj7h0000gn/T/ipykernel_8037/2197572063.py:2: RuntimeWarning: coroutine 'Flow.serve' was never awaited
  workflow.serve(
